<a href="https://colab.research.google.com/github/Taeho-Kim-0322/Deep_Learning_Start/blob/master/%EC%8B%A4%EC%8A%B5_3_3_RNN_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_%EC%8B%AC%ED%99%94%EB%AA%A8%EB%8D%B8%EB%A7%81_%EA%B9%80%ED%83%9C%ED%98%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습 3. RNN을 이용한 😀감정분석😑 모델 학습하기



<b>학습 목표:    
- LSTM, GRU 등 다양한 RNN 계열 셀들을 활용해본다.
- Bidirectional RNN, Multi-layer RNN, 모델 앙상블을 모델링해본다.
</b>








## #0. 실습 준비하기
지난 실습에서는 SimpleRNN을 사용해 감성분석 모델링을 진행했습니다.    
이번 시간에는 이론으로 학습한 다양한 셀 구조와 모델 아키텍처를 사용해 모델링을 진행해보겠습니다.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
## train, validation, test 데이터 로딩
!cp "/content/gdrive/My Drive/NLP/utils.py" "/content"

import pickle
import numpy as np
with open("/content/gdrive/My Drive/NLP/Sentiment_prepro_data.pkl", "rb") as f:
  prepro_data = pickle.load(f)
train_ids = prepro_data["train_ids"]
train_labels = prepro_data["train_labels"]
val_ids = prepro_data["val_ids"]
val_labels = prepro_data["val_labels"]
test_ids = prepro_data["test_ids"]
test_labels = prepro_data["test_labels"]
label_map = prepro_data["label_map"]
print(len(train_ids), len(train_labels), len(val_ids), len(val_labels), len(test_ids), len(test_labels))

49998 49998 10000 10000 10000 10000


In [5]:
## 단어사전 & text_encoder 로딩
from utils import TextEncoder
import json
with open("/content/gdrive/My Drive/NLP/Sentiment_vocab.json", "r") as f:
  new_vocab_list = json.loads(f.read())

text_encoder = TextEncoder(new_vocab_list)

In [7]:
""" CBOW 워드벡터 로딩 """

## final_embeddings: 70002개 토큰에 대한 워드 벡터 매트릭스 shape=(70002, 128)

with open("/content/gdrive/My Drive/NLP/vecs.tsv") as f:
  vecs = [v.strip() for v in f.readlines()]
  final_embeddings = [v.split("\t") for v in vecs]
  final_embeddings = np.array(final_embeddings, dtype="float32")

## #1. 모델링 실습

### MODEL1: LSTM 셀 사용하기

In [9]:
import tensorflow as tf
tf.keras.backend.clear_session()

LSTM 셀은 tensorflow.keras.layers에 있는 LSTM 레이어를 사용하면 됩니다.   
사용하는 방법은 SimpleRNN과 동일합니다.

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense

vocab_size = text_encoder.vocab_size # 단어사전 개수
embedding_dim = final_embeddings.shape[1] # 임베딩 차원
rnn_hidden_dim = 50 # GRU hidden_size
final_dim = len(label_map)

""" MAKE MODEL """
model1 = Sequential(
    [Embedding(vocab_size, embedding_dim, mask_zero=True),
     LSTM(rnn_hidden_dim),
     Dense(rnn_hidden_dim, activation= "relu"),
     Dense(2, activation="softmax")]
)

In [11]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         10987392  
_________________________________________________________________
lstm (LSTM)                  (None, 50)                35800     
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 102       
Total params: 11,025,844
Trainable params: 11,025,844
Non-trainable params: 0
_________________________________________________________________


In [12]:
"""CBOW로 학습된 워드 임베딩을 Initialize 해주기"""
import random
org_vocab_size = final_embeddings.shape[0]
rand_initial = np.random.uniform(-1,1,size=[vocab_size-org_vocab_size,embedding_dim])
# CBOW 학습된 임베딩 + 랜덤 initialize한 weight를 모델의 weight에 대입
initial_weight = np.append(final_embeddings, rand_initial, axis = 0)
model1.weights[0].assign(initial_weight)

<tf.Variable 'UnreadVariable' shape=(85839, 128) dtype=float32, numpy=
array([[-0.01368477, -0.02725928, -0.02415868, ..., -0.00882053,
        -0.0210129 ,  0.04017795],
       [-0.18259747,  0.13225454, -0.40374675, ..., -0.24515007,
         0.5843183 ,  0.41799858],
       [-0.21374829,  0.17352384,  0.4282032 , ...,  0.8664426 ,
        -0.13740769, -0.5702798 ],
       ...,
       [ 0.6875193 ,  0.41672578, -0.75989676, ...,  0.88178116,
         0.548113  , -0.92541224],
       [ 0.56615865,  0.02021094,  0.10119262, ..., -0.22557952,
        -0.41045088,  0.41594702],
       [ 0.69223934,  0.9076332 ,  0.3547939 , ...,  0.84678483,
        -0.28919587,  0.42994663]], dtype=float32)>

In [13]:
## 모델 컴파일
model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
## 모델 학습
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=1)

num_epochs = 5
history = model1.fit(train_ids, train_labels, epochs=num_epochs, batch_size=200,
                    validation_data=(val_ids, val_labels), callbacks=[callback])

Epoch 1/5
250/250 [==============================] - 17s 67ms/step - loss: 0.4986 - accuracy: 0.7465 - val_loss: 0.3901 - val_accuracy: 0.8263
Epoch 2/5
250/250 [==============================] - 16s 64ms/step - loss: 0.3434 - accuracy: 0.8497 - val_loss: 0.3880 - val_accuracy: 0.8258
Epoch 3/5
250/250 [==============================] - 16s 63ms/step - loss: 0.2806 - accuracy: 0.8843 - val_loss: 0.3824 - val_accuracy: 0.8371
Epoch 4/5
250/250 [==============================] - 16s 64ms/step - loss: 0.2342 - accuracy: 0.9065 - val_loss: 0.4064 - val_accuracy: 0.8325


In [15]:
## 테스트 데이터에 대해 성능 평가
model1.evaluate(test_ids, test_labels)

313/313 [==============================] - 1s 4ms/step - loss: 0.4055 - accuracy: 0.8306


[0.4054609537124634, 0.8306000232696533]

### MODEL2: Bi-LSTM 모델 만들기

Bi-RNN 모델은 keras.layers의 Bidirectional Layer로 RNN계열 레이어를 감싸서 코딩할 수 있습니다. 

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Bidirectional

vocab_size = text_encoder.vocab_size # 단어사전 개수
embedding_dim = final_embeddings.shape[1] # 임베딩 차원
rnn_hidden_dim = 50 # GRU hidden_size
final_dim = len(label_map)

""" MAKE MODEL """
model2 = Sequential(
    [Embedding(vocab_size, embedding_dim, mask_zero=True),
     Bidirectional(LSTM(rnn_hidden_dim)),
     Dense(rnn_hidden_dim, activation= "relu"),
     Dense(2, activation="softmax")]
)

In [17]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         10987392  
_________________________________________________________________
bidirectional (Bidirectional (None, 100)               71600     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102       
Total params: 11,064,144
Trainable params: 11,064,144
Non-trainable params: 0
_________________________________________________________________


👉bidirectional 레이어를 타고 나온 hidden vector의 차원이 100차원인 것을 확인할 수 있습니다.   
orward LSTM에서 나온 50차원의 벡터와 backward LSTM에서 나온 50차원의 벡터를 concatenate했기 때문입니다.

In [18]:
model2.weights[0].assign(initial_weight)

<tf.Variable 'UnreadVariable' shape=(85839, 128) dtype=float32, numpy=
array([[-0.01368477, -0.02725928, -0.02415868, ..., -0.00882053,
        -0.0210129 ,  0.04017795],
       [-0.18259747,  0.13225454, -0.40374675, ..., -0.24515007,
         0.5843183 ,  0.41799858],
       [-0.21374829,  0.17352384,  0.4282032 , ...,  0.8664426 ,
        -0.13740769, -0.5702798 ],
       ...,
       [ 0.6875193 ,  0.41672578, -0.75989676, ...,  0.88178116,
         0.548113  , -0.92541224],
       [ 0.56615865,  0.02021094,  0.10119262, ..., -0.22557952,
        -0.41045088,  0.41594702],
       [ 0.69223934,  0.9076332 ,  0.3547939 , ...,  0.84678483,
        -0.28919587,  0.42994663]], dtype=float32)>

In [19]:
## 모델 컴파일
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
## 모델 학습
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=1)

num_epochs = 5
history = model2.fit(train_ids, train_labels, epochs=num_epochs, batch_size=200,
                    validation_data=(val_ids, val_labels), callbacks=[callback])

Epoch 1/5
250/250 [==============================] - 19s 75ms/step - loss: 0.4983 - accuracy: 0.7442 - val_loss: 0.3907 - val_accuracy: 0.8243
Epoch 2/5
250/250 [==============================] - 17s 70ms/step - loss: 0.3359 - accuracy: 0.8542 - val_loss: 0.3755 - val_accuracy: 0.8275
Epoch 3/5
250/250 [==============================] - 17s 69ms/step - loss: 0.2713 - accuracy: 0.8873 - val_loss: 0.3927 - val_accuracy: 0.8190


In [21]:
## 테스트 데이터에 대해 성능 평가
model2.evaluate(test_ids, test_labels)

313/313 [==============================] - 2s 5ms/step - loss: 0.3884 - accuracy: 0.8246


[0.38835659623146057, 0.8245999813079834]

### MODEL3: Multi-layer-LSTM 모델 만들기

Multi-layer RNN 모델을 만들기 위해서는 하단의 RNN 레이어에서 return_sequences 옵션을 True로 설정해야 합니다.   
다음 레이어에서는 이전 레이어에서 반환한 시퀀스 hidden state를 인풋으로 받기 때문입니다.   

In [22]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout

vocab_size = text_encoder.vocab_size # 단어사전 개수
embedding_dim = final_embeddings.shape[1] # 임베딩 차원
rnn_hidden_dim = 50 # GRU hidden_size
final_dim = len(label_map)

""" MAKE MODEL """
model3 = Sequential(
    [Embedding(vocab_size, embedding_dim, mask_zero=True),
     GRU(rnn_hidden_dim, return_sequences = True),
     Dropout(0.2),
     LSTM(rnn_hidden_dim, return_sequences = False),
     Dense(2, activation="softmax")]
)

In [23]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         10987392  
_________________________________________________________________
gru (GRU)                    (None, None, 50)          27000     
_________________________________________________________________
dropout (Dropout)            (None, None, 50)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 102       
Total params: 11,034,694
Trainable params: 11,034,694
Non-trainable params: 0
_________________________________________________________________


In [24]:
model3.weights[0].assign(initial_weight)

<tf.Variable 'UnreadVariable' shape=(85839, 128) dtype=float32, numpy=
array([[-0.01368477, -0.02725928, -0.02415868, ..., -0.00882053,
        -0.0210129 ,  0.04017795],
       [-0.18259747,  0.13225454, -0.40374675, ..., -0.24515007,
         0.5843183 ,  0.41799858],
       [-0.21374829,  0.17352384,  0.4282032 , ...,  0.8664426 ,
        -0.13740769, -0.5702798 ],
       ...,
       [ 0.6875193 ,  0.41672578, -0.75989676, ...,  0.88178116,
         0.548113  , -0.92541224],
       [ 0.56615865,  0.02021094,  0.10119262, ..., -0.22557952,
        -0.41045088,  0.41594702],
       [ 0.69223934,  0.9076332 ,  0.3547939 , ...,  0.84678483,
        -0.28919587,  0.42994663]], dtype=float32)>

In [25]:
## 모델 컴파일
model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
## 모델 학습
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=1)

num_epochs = 5
history = model3.fit(train_ids, train_labels, epochs=num_epochs, batch_size=200,
                    validation_data=(val_ids, val_labels), callbacks=[callback])

Epoch 1/5
250/250 [==============================] - 19s 76ms/step - loss: 0.4947 - accuracy: 0.7507 - val_loss: 0.3903 - val_accuracy: 0.8230
Epoch 2/5
250/250 [==============================] - 17s 70ms/step - loss: 0.3438 - accuracy: 0.8487 - val_loss: 0.3699 - val_accuracy: 0.8352
Epoch 3/5
250/250 [==============================] - 17s 70ms/step - loss: 0.2832 - accuracy: 0.8821 - val_loss: 0.3836 - val_accuracy: 0.8392


In [27]:
## 테스트 데이터에 대해 성능 평가
model3.evaluate(test_ids, test_labels)

313/313 [==============================] - 2s 5ms/step - loss: 0.3896 - accuracy: 0.8353


[0.389629602432251, 0.8353000283241272]

### MODEL4: 세 모델의 결과 앙상블하기
마지막으로 위에서 학습한 세 모델을 앙상블하는 코드입니다.   
세 개의 모델을 독립적으로 학습한 후 결과를 앙상블하면 정확도를 높일 수 있습니다. 

In [28]:
def predict(test_ids):
  res1 = model1.predict(test_ids)
  res2 = model2.predict(test_ids)
  res3 = model3.predict(test_ids)
  result = (res1 + res2 + res3) / 3
  return result

In [29]:
prediction = predict(test_ids)
prediction

array([[0.02071741, 0.97928256],
       [0.8880331 , 0.11196694],
       [0.99590826, 0.00409172],
       ...,
       [0.03268683, 0.96731323],
       [0.995199  , 0.00480097],
       [0.8503258 , 0.14967419]], dtype=float32)

👉predict 함수는 세 모델이 예측한 결과를 평균한 확률값을 아웃풋으로 반환합니다.    

최종적으로 예측을 하기 위해서는 이 확률값을 카테고리로 변경해야 하겠지요?   
np.argmax 함수는 주어진 축에 대해 최대값의 위치를 찾아주는 함수입니다.   
이 함수를 사용해 확률값이 가장 높은 카테고리를 모델 예측치로 사용할 수 있습니다.   


In [30]:
""" catecory로 변경 """
prediction = np.argmax(prediction, axis = 1)

In [31]:
print("TEST ACCURACY:")
sum(prediction == test_labels) / len(test_labels)

TEST ACCURACY:


0.8404

---

## #2. DAILY MISSION 🙌

아래의 세 모델은 RNN을 사용하여 만든 감성분석 모델입니다.   
그런데 무슨 문제인지, 학습이 잘 이루어지지 않고 있습니다.   
모델을 살펴보고, 어떤 오류가 있는지 찾아 디버깅한 후 파일을 제출해주세요!

#### model_1

In [33]:
tf.keras.backend.clear_session()

vocab_size = text_encoder.vocab_size # 단어사전 개수
embedding_dim = final_embeddings.shape[1] # 임베딩 차원
rnn_hidden_dim = 50 # GRU hidden_size

""" MAKE MODEL """
model_1 = Sequential(
    [Embedding(vocab_size, embedding_dim, mask_zero = True),
     GRU(rnn_hidden_dim),
     Dense(2, activation = 'softmax')]
)
model_1.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         10987392  
_________________________________________________________________
gru (GRU)                    (None, 50)                27000     
_________________________________________________________________
dense (Dense)                (None, 2)                 102       
Total params: 11,014,494
Trainable params: 11,014,494
Non-trainable params: 0
_________________________________________________________________


- 오류가 있는 부분: 
  1. 패딩된 부분 마스킹이 필요하기 때문에, mask_zero = True 추가
  2. 마지막 Dense에는 activation 추가가 필요함

In [34]:
model_1.weights[0].assign(initial_weight)
## 모델 컴파일
model_1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## 모델 학습
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=1)

num_epochs = 5
history = model_1.fit(train_ids, train_labels, epochs=num_epochs, batch_size=200,
                    validation_data=(val_ids, val_labels), callbacks=[callback])

Epoch 1/5
250/250 [==============================] - 18s 71ms/step - loss: 0.5178 - accuracy: 0.7318 - val_loss: 0.4034 - val_accuracy: 0.8150
Epoch 2/5
250/250 [==============================] - 17s 68ms/step - loss: 0.3511 - accuracy: 0.8474 - val_loss: 0.3837 - val_accuracy: 0.8256
Epoch 3/5
250/250 [==============================] - 17s 68ms/step - loss: 0.2896 - accuracy: 0.8803 - val_loss: 0.3858 - val_accuracy: 0.8226


#### model_2

In [35]:
tf.keras.backend.clear_session()

vocab_size = text_encoder.vocab_size # 단어사전 개수
embedding_dim = final_embeddings.shape[1] # 임베딩 차원
rnn_hidden_dim = 50 # GRU hidden_size

""" MAKE MODEL """
model_2 = Sequential(
    [Embedding(vocab_size, embedding_dim, mask_zero = True),
     GRU(rnn_hidden_dim, return_sequences=True),
     GRU(rnn_hidden_dim, return_sequences=False),
     Dense(2, activation="softmax")]
)
model_2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         10987392  
_________________________________________________________________
gru (GRU)                    (None, None, 50)          27000     
_________________________________________________________________
gru_1 (GRU)                  (None, 50)                15300     
_________________________________________________________________
dense (Dense)                (None, 2)                 102       
Total params: 11,029,794
Trainable params: 11,029,794
Non-trainable params: 0
_________________________________________________________________


- 오류가 있는 부분: 최종 GRU 다음에는 RNN 레이어가 없기 때문에, return_sequences = False로 수정

In [36]:
model_2.weights[0].assign(initial_weight)
## 모델 컴파일
model_2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## 모델 학습
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=1)

num_epochs = 5
history = model_2.fit(train_ids, train_labels, epochs=num_epochs, batch_size=200,
                    validation_data=(val_ids, val_labels), callbacks=[callback])

Epoch 1/5
250/250 [==============================] - 19s 76ms/step - loss: 0.4971 - accuracy: 0.7491 - val_loss: 0.4032 - val_accuracy: 0.8218
Epoch 2/5
250/250 [==============================] - 18s 71ms/step - loss: 0.3445 - accuracy: 0.8520 - val_loss: 0.3739 - val_accuracy: 0.8345
Epoch 3/5
250/250 [==============================] - 18s 71ms/step - loss: 0.2829 - accuracy: 0.8830 - val_loss: 0.3833 - val_accuracy: 0.8387


#### model_3

In [37]:
tf.keras.backend.clear_session()

vocab_size = text_encoder.vocab_size # 단어사전 개수
embedding_dim = final_embeddings.shape[1] # 임베딩 차원
rnn_hidden_dim = 50 # GRU hidden_size

""" MAKE MODEL """
model_3 = Sequential(
    [Embedding(vocab_size, embedding_dim, mask_zero = True),
     LSTM(rnn_hidden_dim),
     Dense(2, activation="softmax")]
)
model_3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         10987392  
_________________________________________________________________
lstm (LSTM)                  (None, 50)                35800     
_________________________________________________________________
dense (Dense)                (None, 2)                 102       
Total params: 11,023,294
Trainable params: 11,023,294
Non-trainable params: 0
_________________________________________________________________


- 오류가 있는 부분: 최종 Dense 레이어의 shape 값이 2가 되어야 함

In [38]:
model_3.weights[0].assign(initial_weight)
## 모델 컴파일
model_3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## 모델 학습
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=1)

num_epochs = 5
history = model_3.fit(train_ids, train_labels, epochs=num_epochs, batch_size=200,
                    validation_data=(val_ids, val_labels), callbacks=[callback])

Epoch 1/5
250/250 [==============================] - 17s 69ms/step - loss: 0.4977 - accuracy: 0.7476 - val_loss: 0.4025 - val_accuracy: 0.8136
Epoch 2/5
250/250 [==============================] - 16s 65ms/step - loss: 0.3444 - accuracy: 0.8493 - val_loss: 0.3776 - val_accuracy: 0.8263
Epoch 3/5
250/250 [==============================] - 16s 66ms/step - loss: 0.2798 - accuracy: 0.8835 - val_loss: 0.3886 - val_accuracy: 0.8276
